# Training Models

**NOTES**

- In Linear Regression, Gradient Descendent is guaranteed to find the global minimum, if you wait long enough and if the learning rate is not too high.

- When using Gradient Descendent, you need to ensure all features have a similar scale, or it willl take long to converge.

- Training a model with thousands of features is much faster if you use Gradient Descendent than the Normal Equation or SVD composition as cost function.

- When setting the number of steps (iterations), a simple way to go is to set a very large number, but at the same time set a tolerance, so if the algorithm reaches a tolerance number (the cost is not changing much anymore), you stop running the iterations. This saves computational time, while ensuring you have a good solution.

- **Stochastic Gradient Descendent** (SGD) does not use all the training data, but it runs in batches. This can be good to jump from local minumuns in search for a global minimum. 

-- **Ridge (L2)** uses alpha as a regularisation parameter. If alpha =0, there's no regularisation (it's just a linear model), if it's high, weights will be closer to zero (less weight to the features). *YOU NEED TO SCALE THE DATA*. It's doesn't touch the intercept (bias term).
Higher alphas lead to more smooth, almost flat lines when Ridge is used with PolynomialFeatures.


- **Lasso (L1)** have the tendency to eliminate features with least importance; it automatically does feature selection. 

- **ElasticNet** is a mix of Ridge and Lasso. When r=0, we have Ridge, when it's r=1 we have Lasso.

- You can use SGD, with penalty=L1 or L2.

**- Usually it's better to use LinearRegression with some form of regularization. L2 is like a default, but if you suspect many features do no matter, use L2. In fact, ElasticNet is a lot of times preferred, as L2 alone can be too erratic.**


<img src="../img/al_lin_reg.png" width="60%">

- If you plot the Learning Curve of your model you can have an idea if it's perfoming well and not under/over fitting. Train the model several times on different sized subsets of the training data

In [ ]:
# plotting learning curve

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_erros = [], []

    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predic = model.predict(X_train[:m])
        y_val_predic = model.predict(X_val)

        train_errors.append(mean_squared_error(y_train_predic, y_train[:m]))
        val_erros.append(mean_squared_error(y_val, y_val_pred))

    # plot
    plt.plot(np.sqrt(train_errors), label="train")
    plt.plot(np.sqrt(val_erros), label="validation")

- It's common for a model to use a different function for training and another one for evaluating. This is because for training the function needs to have a derivative, and the metric for evaluation is connected to the problem. An example here is a classifier which uses **log loss** as a cost function, but **accuracy / precision / recall** for evaluation.

## 1.0 Early Stopping

With early stopping you don't let the model run the "perfect solution"but you stop it when it reaches a certain threshold.

In [ ]:
# basic implementation of early stopping
from sklearn.base import clone

#prepare the data
poly_scaler = Pipeline([
    ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
    ("standard_scaler", StandardScaler())
])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scale = poly_scaler.transform(X_validation)

sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True, penalty=None, learning_rate="constant", eta0=0.0005)
# EXPLANATION OF THE PARAMETERS
# max_iter=1 because it will run one time. We will control the epochs with the loop below
# tol=np.infty is the tolerance for stopping the iterations. When set to np.infty it will force to run with the max_iter
# warm_start=True for next fittings, it starts from where it left off, previous fittings
# learning_rate will remain constant during the fitting. 
# sometimes you might want to start with a large learning rate and reduce it with the number of fittings. this helps the model to scape valleys and then reach the global minimum
# eta0 sets the initial learning rate

min_val_error = float("inf")
best_epoch = None
best_model = None

for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train) # continues where it left off
    y_val_pred = sgd_reg.predict(X_val_poly_scale)
    val_error = mean_squared_error(y_val_pred, y_val)

    if val_err < min_val_error:
        min_val_error = val_error
        best_epoch = epoch
        best_model = clone(sgd_reg)



## 2.0 Logistic Regression

-- For logreg, **.predict_proba** shows the probability of a certain item to be in respective classes, while **.pred** gives you the class more likely it belongs to.

-- The hyperparameter controlling the regularization strength of a Scikit-Learn LogisticRegression model is not alpha (as in other
linear models), but its inverse: C. The higher the value of C, the less the model is regularized.

-- Logit is the ratio between the *(probability of being positive) / (probability of being negative)*

-- Softmax regressor can only be used when there's only one class the instance belongs to (ie, it can identify one class of flower among n classes, but it can't identify multiple people in one picture)

-- Entropy says how much your probability distribution is likely to change.

-- LogisticRegression in Scikit-Learn uses "One-Versus-All" by default, but you can adjust the hyperparameter to "multinomial"to switch it to Softmax Regression.


-- **Binary Classification: Use either Logistic Regression or Softmax (they're equivalent).
Multiclass Classification: Use Softmax.**

In [ ]:
# using softmax in Scikit-Learn

softmax_regressor = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)
# we speficify C because by default it uses penalty=L2, so we can use C as it's inverse and tweak that

# Linear Regression vs. SGDRegressor: A Comparison

Both Linear Regression and SGDRegressor are used for linear regression tasks, but they have different approaches and use cases:

## Linear Regression:

Batch Gradient Descent: Uses the entire dataset to calculate the gradient in each iteration.
Suitable for: Smaller datasets or when computational resources are not a constraint.
Convergence: Generally converges to a global minimum.

## SGDRegressor:

Stochastic Gradient Descent: Uses a single random sample from the dataset to calculate the gradient in each iteration.
Suitable for: Larger datasets or online learning scenarios where data arrives sequentially.
Convergence: May not always converge to the global minimum, but it can be faster for large datasets.
When to Use Which:

**Linear Regression:**
If your dataset is relatively small and you have sufficient computational resources.
If you prioritize global convergence and accuracy.

**SGDRegressor:**
If your dataset is large and you need to train the model incrementally.
If you want to handle streaming data or online learning scenarios.
If you're willing to sacrifice some accuracy for faster training.


**Key Considerations:**

Learning Rate: The learning rate in SGDRegressor affects convergence speed and accuracy. A smaller learning rate might lead to slower convergence but better accuracy.
Regularization: Both Linear Regression and SGDRegressor can use regularization techniques (e.g., L1 or L2) to prevent overfitting.
Randomness: SGDRegressor introduces randomness due to the stochastic nature of the gradient calculation. This can affect the convergence path and final solution.

# REVIEW

-- If you have a training set with millions of features you can use Stochastic Gradient Descent or Mini-batch Gradient Descent, and perhaps Batch Gradient Descent if the training set fits in memory. But you cannot use the Normal Equation or the SVD approach because the computational complexity grows quickly (more than quadratically) with the number of features.

-- If the features in your training set have very different scales, the cost function will have the shape of an elongated bowl, so the Gradient Descent algorithms will take a long time to converge. To solve this you should scale the data before training the model. Note that the Normal Equation or SVD approach will work just fine without scaling. Moreover, regularized models may converge to a suboptimal solution if the features are not scaled: since regularization penalizes large weights, features with smaller values will tend to be ignored compared to features with larger values.

-- Gradient Descent cannot get stuck in a local minimum when training a Logistic Regression model because the cost function is convex. Convex means that if you draw a straight line between any two points on the curve, the line never crosses the curve.
If the optimization problem is convex (such as Linear Regression or Logistic Regression), and assuming the learning rate is not too high, then all Gradient Descent algorithms will approach the global optimum and end up producing fairly similar models. However, unless you gradually reduce the learning rate, Stochastic GD and Mini-batch GD will never truly converge; instead, they will keep jumping back and forth around the global optimum. This means that even if you let them run for a very long time, these Gradient Descent algorithms will produce slightly different models.

-- If the validation error consistently goes up after every epoch, then one possibility is that the learning rate is too high and the algorithm is diverging. If the training error also goes up, then this is clearly the problem and you should reduce the learning rate. However, if the training error is not going up, then your model is overfitting the training set and you should stop training.

-- Due to their random nature, neither Stochastic Gradient Descent nor Mini-batch Gradient Descent is guaranteed to make progress at every single training iteration. So if you immediately stop training when the validation error goes up, you may stop much too early, before the optimum is reached. A better option is to save the model at regular intervals; then, when it has not improved for a long time (meaning it will probably never beat the record), you can revert to the best saved model.

-- Stochastic Gradient Descent has the fastest training iteration since it considers only one training instance at a time, so it is generally the first to reach the vicinity of the global optimum (or Mini-batch GD with a very small mini-batch size). However, only Batch Gradient Descent will actually converge, given enough training time. As mentioned, Stochastic GD and Mini-batch GD will bounce around the optimum, unless you gradually reduce the learning rate.

-- If the validation error is much higher than the training error, this is likely because your model is overfitting the training set. One way to try to fix this is to reduce the polynomial degree: a model with fewer degrees of freedom is less likely to overfit. Another thing you can try is to regularize the model—for example, by adding an ℓ₂ penalty (Ridge) or an ℓ₁ penalty (Lasso) to the cost function. This will also reduce the degrees of freedom of the model. Lastly, you can try to increase the size of the training set.

-- If both the training error and the validation error are almost equal and fairly high, the model is likely underfitting the training set, which means it has a high bias. You should try reducing the regularization hyperparameter α.


-- A model with some regularization typically performs better than a model without any regularization, so you should generally prefer Ridge Regression over plain Linear Regression.
Lasso Regression uses an ℓ₁ penalty, which tends to push the weights down to exactly zero. This leads to sparse models, where all weights are zero except for the most important weights. This is a way to perform feature selection automatically, which is good if you suspect that only a few features actually matter. When you are not sure, you should prefer Ridge Regression.
Elastic Net is generally preferred over Lasso since Lasso may behave erratically in some cases (when several features are strongly correlated or when there are more features than training instances). However, it does add an extra hyperparameter to tune. If you want Lasso without the erratic behavior, you can just use Elastic Net with an l1_ratio close to 1.

-- If you want to classify pictures as outdoor/indoor and daytime/nighttime, since these are not exclusive classes (i.e., all four combinations are possible) you should train two Logistic Regression classifiers.

